In [28]:
#instead of one-hot encoding, use 1,-1 on single node
#use all data of day as one vector

import pandas as pd
import csv

df = pd.read_csv('raw/삼성전자.csv').drop('Date', axis=1)
iter_len = len(df['Close'])
filename = 'prep1/삼성전자.csv'


# Labeling: increase = 1, decrease = -1

with open(filename, 'w', newline='') as f:
    wt = csv.writer(f)
    wt.writerow("Open,High,Low,Close,Adj Close,Volume".split(",")+['Label'])
    for i in range(iter_len-1):
        line1 = df.iloc[i]
        line2 = df.loc[i+1]
        label = 1 if line2['Adj Close']>line1['Adj Close'] else -1
        line1['Label']=label
        wt.writerow(line1.tolist())

<ipython-input-28-72ca8e9c73a0>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  line1['Label']=label
C:\Users\juni\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [29]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F


f = pd.read_csv(filename)
labels = f.axes[1]
rawX = f.drop(columns=labels[-1], axis=1).values
rawY = f.drop(columns=labels[:-1], axis=1).values

In [30]:
newY = []
empty = np.zeros((1,), dtype=int)

for val in rawY:
    row = empty.copy()
    row[0]=val
    newY.append(row)

x=np.array(rawX)
y=np.array(newY)

if torch.cuda.is_available():
    x=torch.tensor(x).cuda()
    y=torch.tensor(y).cuda()
    print("CUDA available")
else:
    x=torch.tensor(x)
    y=torch.tensor(y)
    

In [31]:
class Net(torch.nn.Module):
    def __init__(self, D_in, H, D_out):
        super(Net, self).__init__()
        self.l1 = nn.Linear(D_in, H)
        self.h1 = nn.Linear(H, 2*H)
        self.h2 = nn.Linear(2*H, 4*H)
        self.l2 = nn.Linear(4*H, D_out)

    def forward(self, x):
        x = F.relu(self.l1(x))
        x = F.relu(self.h1(x))
        x = F.relu(self.h2(x))
        y = F.relu(self.l2(x))
        y = torch.sign(y)
        return y


D_in, D_out = 6, 1 #input, output 

def run(H, lr, epochs):
  
    #training
    
    model = Net(D_in, H, D_out)
    if torch.cuda.is_available(): model = model.cuda()
    criterion = torch.nn.MSELoss(reduction='sum')
    optimizer = torch.optim.SGD(model.parameters(), lr=lr)
    for t in range(epochs):
        y_pred = model(x.float())
        loss = criterion(y_pred, y.float())
        print(t, loss.item())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

  ###############################################################################
  
    #evaluation
    total=0
    correct=0
    
    for idx, inp in enumerate(x):
        total+=1
        y_pred = model(inp.float()).cpu().detach().numpy()
        y_truth = list(newY[idx])
        print(y_pred, y_truth)
        if y_pred == y_truth: correct+=1
    
    print(round(100*correct/total,2))


In [32]:
run(12, 1e-3, 100)

0 11217.0
1 11217.0
2 11217.0
3 11217.0
4 11217.0
5 11217.0
6 11217.0
7 11217.0
8 11217.0
9 11217.0
10 11217.0
11 11217.0
12 11217.0
13 11217.0
14 11217.0
15 11217.0
16 11217.0
17 11217.0
18 11217.0
19 11217.0
20 11217.0
21 11217.0
22 11217.0
23 11217.0
24 11217.0
25 11217.0
26 11217.0
27 11217.0
28 11217.0
29 11217.0
30 11217.0
31 11217.0
32 11217.0
33 11217.0
34 11217.0
35 11217.0
36 11217.0
37 11217.0
38 11217.0
39 11217.0
40 11217.0
41 11217.0
42 11217.0
43 11217.0
44 11217.0
45 11217.0
46 11217.0
47 11217.0
48 11217.0
49 11217.0
50 11217.0
51 11217.0
52 11217.0
53 11217.0
54 11217.0
55 11217.0
56 11217.0
57 11217.0
58 11217.0
59 11217.0
60 11217.0
61 11217.0
62 11217.0
63 11217.0
64 11217.0
65 11217.0
66 11217.0
67 11217.0
68 11217.0
69 11217.0
70 11217.0
71 11217.0
72 11217.0
73 11217.0
74 11217.0
75 11217.0
76 11217.0
77 11217.0
78 11217.0
79 11217.0
80 11217.0
81 11217.0
82 11217.0
83 11217.0
84 11217.0
85 11217.0
86 11217.0
87 11217.0
88 11217.0
89 11217.0
90 11217.0
91 11217.

[1.] [1]
[1.] [-1]
[1.] [-1]
[1.] [1]
[1.] [-1]
[1.] [-1]
[1.] [1]
[1.] [-1]
[1.] [-1]
[1.] [1]
[1.] [1]
[1.] [-1]
[1.] [1]
[1.] [1]
[1.] [-1]
[1.] [1]
[1.] [-1]
[1.] [1]
[1.] [1]
[1.] [-1]
[0.] [1]
[1.] [-1]
[1.] [1]
[1.] [1]
[1.] [1]
[1.] [-1]
[1.] [1]
[1.] [-1]
[1.] [1]
[1.] [-1]
[1.] [-1]
[1.] [-1]
[1.] [1]
[1.] [1]
[1.] [-1]
[1.] [-1]
[1.] [1]
[1.] [1]
[1.] [1]
[1.] [1]
[1.] [1]
[1.] [1]
[1.] [-1]
[1.] [-1]
[1.] [1]
[1.] [1]
[1.] [-1]
[1.] [1]
[1.] [-1]
[0.] [-1]
[1.] [1]
[1.] [1]
[1.] [-1]
[1.] [1]
[1.] [1]
[1.] [1]
[1.] [1]
[1.] [-1]
[1.] [-1]
[1.] [1]
[1.] [-1]
[1.] [1]
[1.] [-1]
[1.] [-1]
[1.] [-1]
[1.] [1]
[1.] [-1]
[1.] [1]
[1.] [1]
[1.] [-1]
[0.] [-1]
[1.] [1]
[1.] [1]
[1.] [1]
[1.] [1]
[1.] [-1]
[1.] [1]
[1.] [-1]
[1.] [-1]
[1.] [-1]
[1.] [1]
[1.] [-1]
[1.] [1]
[1.] [1]
[1.] [1]
[1.] [1]
[1.] [1]
[1.] [-1]
[0.] [-1]
[0.] [-1]
[0.] [-1]
[1.] [1]
[1.] [1]
[1.] [-1]
[1.] [-1]
[1.] [-1]
[1.] [-1]
[1.] [1]
[1.] [-1]
[1.] [-1]
[1.] [-1]
[1.] [1]
[1.] [1]
[1.] [1]
[1.] [-1]
[0.] 

[1.] [-1]
[1.] [1]
[1.] [1]
[1.] [1]
[1.] [-1]
[1.] [1]
[1.] [1]
[1.] [-1]
[1.] [-1]
[1.] [1]
[1.] [1]
[1.] [-1]
[1.] [-1]
[1.] [1]
[1.] [-1]
[1.] [1]
[1.] [-1]
[1.] [-1]
[1.] [-1]
[1.] [1]
[1.] [1]
[1.] [-1]
[1.] [-1]
[1.] [1]
[1.] [1]
[1.] [-1]
[1.] [-1]
[1.] [1]
[1.] [1]
[1.] [1]
[1.] [-1]
[1.] [-1]
[1.] [1]
[1.] [1]
[1.] [1]
[1.] [1]
[1.] [1]
[1.] [-1]
[1.] [1]
[1.] [1]
[1.] [-1]
[1.] [1]
[1.] [1]
[1.] [1]
[1.] [1]
[1.] [1]
[1.] [-1]
[1.] [-1]
[1.] [-1]
[1.] [1]
[1.] [1]
[1.] [-1]
[1.] [-1]
[1.] [-1]
[1.] [1]
[1.] [1]
[1.] [1]
[1.] [1]
[1.] [-1]
[1.] [1]
[1.] [1]
[1.] [1]
[1.] [-1]
[1.] [1]
[1.] [-1]
[0.] [1]
[1.] [-1]
[0.] [1]
[1.] [-1]
[1.] [-1]
[1.] [-1]
[1.] [-1]
[0.] [1]
[1.] [1]
[1.] [1]
[1.] [-1]
[1.] [-1]
[1.] [-1]
[1.] [1]
[1.] [-1]
[1.] [-1]
[1.] [-1]
[1.] [1]
[1.] [-1]
[1.] [1]
[1.] [1]
[1.] [-1]
[1.] [-1]
[1.] [1]
[1.] [-1]
[1.] [-1]
[0.] [-1]
[1.] [-1]
[1.] [1]
[1.] [-1]
[1.] [1]
[1.] [1]
[1.] [-1]
[1.] [1]
[1.] [-1]
[1.] [-1]
[1.] [-1]
[1.] [1]
[1.] [-1]
[1.] [1]
[1.]

[1.] [1]
[1.] [1]
[1.] [1]
[1.] [-1]
[1.] [1]
[1.] [-1]
[1.] [-1]
[1.] [1]
[1.] [-1]
[1.] [1]
[1.] [-1]
[1.] [1]
[1.] [-1]
[1.] [1]
[1.] [1]
[1.] [1]
[1.] [1]
[1.] [-1]
[1.] [-1]
[1.] [1]
[1.] [1]
[1.] [1]
[1.] [-1]
[1.] [-1]
[1.] [1]
[1.] [1]
[1.] [1]
[1.] [1]
[1.] [1]
[1.] [-1]
[1.] [1]
[1.] [-1]
[1.] [1]
[1.] [-1]
[1.] [1]
[1.] [-1]
[1.] [1]
[1.] [-1]
[1.] [1]
[1.] [1]
[1.] [-1]
[1.] [1]
[1.] [1]
[1.] [-1]
[1.] [-1]
[1.] [1]
[1.] [-1]
[1.] [-1]
[1.] [1]
[1.] [-1]
[1.] [1]
[1.] [1]
[1.] [1]
[1.] [-1]
[1.] [-1]
[1.] [-1]
[1.] [-1]
[1.] [-1]
[1.] [1]
[1.] [1]
[1.] [1]
[1.] [-1]
[1.] [-1]
[1.] [-1]
[1.] [1]
[1.] [-1]
[1.] [1]
[1.] [-1]
[1.] [-1]
[1.] [-1]
[1.] [-1]
[1.] [1]
[1.] [1]
[1.] [1]
[1.] [-1]
[1.] [-1]
[1.] [-1]
[1.] [-1]
[1.] [-1]
[1.] [1]
[1.] [1]
[1.] [1]
[1.] [1]
[1.] [1]
[1.] [-1]
[1.] [1]
[1.] [1]
[1.] [1]
[1.] [-1]
[1.] [-1]
[1.] [-1]
[1.] [-1]
[1.] [1]
[1.] [-1]
[1.] [1]
[1.] [1]
[1.] [-1]
[1.] [-1]
[1.] [1]
[1.] [-1]
[1.] [1]
[1.] [-1]
[1.] [-1]
[1.] [-1]
[1.] [1]
[1.]

[1.] [-1]
[1.] [1]
[1.] [1]
[1.] [-1]
[1.] [1]
[1.] [-1]
[1.] [-1]
[1.] [-1]
[1.] [-1]
[1.] [-1]
[1.] [-1]
[1.] [1]
[1.] [1]
[1.] [1]
[1.] [-1]
[1.] [-1]
[1.] [-1]
[1.] [1]
[1.] [-1]
[1.] [-1]
[1.] [-1]
[1.] [1]
[1.] [1]
[1.] [1]
[1.] [-1]
[1.] [-1]
[1.] [1]
[1.] [1]
[1.] [1]
[1.] [-1]
[1.] [-1]
[1.] [-1]
[1.] [1]
[1.] [1]
[1.] [1]
[1.] [-1]
[1.] [1]
[1.] [1]
[1.] [1]
[1.] [1]
[1.] [1]
[1.] [-1]
[1.] [1]
[1.] [1]
[1.] [1]
[1.] [1]
[1.] [-1]
[1.] [1]
[1.] [1]
[1.] [1]
[1.] [-1]
[1.] [-1]
[1.] [-1]
[1.] [1]
[1.] [1]
[1.] [1]
[1.] [1]
[1.] [-1]
[1.] [-1]
[1.] [1]
[1.] [-1]
[1.] [-1]
[1.] [1]
[1.] [1]
[1.] [-1]
[1.] [-1]
[1.] [-1]
[1.] [1]
[1.] [1]
[1.] [1]
[1.] [1]
[1.] [-1]
[1.] [-1]
[1.] [-1]
[1.] [-1]
[1.] [1]
[1.] [-1]
[1.] [1]
[1.] [-1]
[1.] [-1]
[1.] [-1]
[1.] [1]
[1.] [-1]
[1.] [1]
[1.] [1]
[1.] [-1]
[1.] [-1]
[1.] [1]
[1.] [-1]
[1.] [-1]
[1.] [1]
[1.] [1]
[1.] [-1]
[1.] [-1]
[1.] [1]
[1.] [1]
[1.] [-1]
[1.] [1]
[1.] [-1]
[1.] [-1]
[1.] [-1]
[1.] [1]
[1.] [1]
[1.] [-1]
[1.] [1]
[1.